In [1]:
from __future__ import division, print_function
import matplotlib.pyplot as plt
import bilby
import numpy as np

import hphc9

C = 299792458.
G = 6.67408*1e-11
Mo = 1.989*1e30
Mpc = 3.086*1e22

%matplotlib inline

In [2]:
outdir = 'outdir_new_model'
label = 'create_your_own_source_model'
sampling_frequency = 4096
duration = 1

In [10]:
def eccentric_waveform(frequency_array_, A, f0, tau, phi0, geocent_time, ra, dec, psi):
    
    N = len(frequency_array_)
    h_plus = np.zeros(N)*1j
    h_cross = np.zeros(N)*1j
    
    k = 0
    for f in frequency_array_:

        arg_plus = {'iota_':tau, 'beta_':psi, 'D_':440.0 , 'm1_':35.0, 'm2_':30.0, 'f_':f, 'f0_':20.0, 'et0_':0.1, 'phic_':phi0, 'tc_':geocent_time}

        fplus = hphc9.Fn(**arg_plus)

        h_plus[k] = fplus.hphc()[0]
        h_cross[k] = fplus.hphc()[1]

        k=k+1
    #return {'plus': h_plus, 'cross': h_cross}
    return {'plus': h_plus, 'cross': h_cross}

In [11]:
import time 


fff = np.arange(0.0,500.0,1.0)
t0 = time.time()
zz = eccentric_waveform(np.arange(0,100,1), A=1e-23, f0=100, tau=1, phi0=0, geocent_time=0,
                            ra=0, dec=0, psi=0)
t1 = time.time()
print('Execution took {:.4f}'.format(t1 - t0))

Execution took 4.0937


In [5]:
# We now define some parameters that we will inject
injection_parameters = dict(A=1e-23, f0=100, tau=1, phi0=0, geocent_time=0,
                            ra=0, dec=0, psi=0)

In [6]:
# Now we pass our source function to the WaveformGenerator
waveform_generator = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    frequency_domain_source_model=eccentric_waveform, parameters=injection_parameters)

14:22 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: __main__.eccentric_waveform
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [7]:
# Set up interferometers.
ifos = bilby.gw.detector.InterferometerList(['H1', 'L1'])
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency, duration=duration,
    start_time=injection_parameters['geocent_time'] - 3)
injection = ifos.inject_signal(waveform_generator=waveform_generator,
                   parameters=injection_parameters)

/home/hemantakumar.phurailatpam/anaconda3/envs/bilby/lib/python3.7/site-packages/bilby/gw/detector/psd.py:356: RuntimeWarning: invalid value encountered in multiply
  frequency_domain_strain = self.__power_spectral_density_interpolated(frequencies) ** 0.5 * white_noise
14:23 bilby WARNING : Injecting signal outside segment, start_time=-3, merger time=0.
14:23 bilby INFO    : Injected signal in H1:
14:23 bilby INFO    :   optimal SNR = 15.60
14:23 bilby INFO    :   matched filter SNR = 16.08+0.06j
14:23 bilby INFO    :   A = 1e-23
14:23 bilby INFO    :   f0 = 100
14:23 bilby INFO    :   tau = 1
14:23 bilby INFO    :   phi0 = 0
14:23 bilby INFO    :   geocent_time = 0
14:23 bilby INFO    :   ra = 0
14:23 bilby INFO    :   dec = 0
14:23 bilby INFO    :   psi = 0
14:23 bilby WARNING : Injecting signal outside segment, start_time=-3, merger time=0.
14:23 bilby INFO    : Injected signal in L1:
14:23 bilby INFO    :   optimal SNR = 24.97
14:23 bilby INFO    :   matched filter SNR = 24.25+0.75